In [2]:
import pandas as pd
import numpy as np
import networkx as nx

#import data
data = pd.read_csv("agaricus-lepiota.data",header=None,na_values=["?",None])

uniqueValues = {}
#this is a file that has all the column names in a csv format and also has an extra "class" at the first column
featureCol = pd.read_csv("agaricus-lepiota.csv")


data.columns = list(featureCol.columns)
data.dropna(inplace = True)
for feature in data.columns:
    uniqueValues[feature] = data[feature].unique()


class Node:
    def __init__(self, feature, children = [], type="split"):
        self.feature = feature
        if isinstance(children, list) == False:
            self.children = [children]
        else:
            self.children = children;
        self.type = type
        
        
def singleLog(num,den):
    if (den == 0 or num == 0):
        return 0
    elif (den == num):
        return 0
    return -(num/den)*np.log2(num/den)

def FeatureEntropy(data,feature):
    uniqueValues = data[feature].unique()
    big_total = len(data.index)
    entropy = 0
    for value in uniqueValues:
        value_in_data = data[data[feature]==value]
        total = len(value_in_data.index)
        num_p = len(value_in_data[value_in_data["class"] == "p"])
        num_e = len(value_in_data[value_in_data["class"] == "e"])
        partialEntropy = singleLog(num_p,total) + singleLog(num_e,total)
        entropy += (total/big_total)*partialEntropy
    return entropy

def trainGreedyDecisionTree(data, current_depth = 0, stopping_depth = float('inf')):
    columns = list(data.columns)
    total_rows = len(data.index)
    num_p = len(data[data["class"] == "p"])
    num_e = len(data[data["class"] == "e"])
    current_entropy = singleLog(num_p,total_rows)+singleLog(num_e,total_rows)
    array_of_ig = {}
    if stopping_depth == 1:
        stopping_depth = 2
    if current_depth >= stopping_depth-1:
        if num_p > num_e:
            return Node("p", [], "output")
        else:
            return Node("e", [], "output")
    #if dataset is empty then return
    if len(data.index) == 0:
        return []
    class_label = data["class"].unique()
    
    #if all class labels are the same then exit recursion and output 
    if len(class_label) == 1:
        return Node(class_label[0],[],"output")


    #if all inputs are the same then exit recursion and output
    if (data.duplicated().sum() == len(data.index)):
        if num_p > num_e:
            return Node("p", [], "output")
        else:
            return Node("e", [], "output")
    
    #calculate largest information gain for each feature
    for feature in columns:
        if feature == "class":
            pass
        else:
            array_of_ig[feature] = current_entropy - FeatureEntropy(data,feature)
    #TODO: change to choosing feature by IG = H(Y) - H(X)
    largest_IG_feature = max(array_of_ig, key=array_of_ig.get)
    #if largest IG feature only has 1 unique feature remaining then it must exit
    #class label will be whatever has majority 
    

    #split the dataset by the features different options 
    child_nodes = []
    #recursively call function and split dataset by the feature options
    largest_features_unique = uniqueValues[largest_IG_feature]
    largest_features_unique_in_dataset = data[largest_IG_feature].unique()
    for value in largest_features_unique:
        #case if a unique value does not exist in the dataset, assume an output of p
        if value in largest_features_unique_in_dataset:
            split_dataset = data[data[largest_IG_feature] == value].drop(largest_IG_feature,axis=1)
            child_nodes.append(Node(value,trainGreedyDecisionTree(split_dataset,current_depth+1,stopping_depth)))
        else:
            child_nodes.append(Node(value, Node("p",[],"output")))
        
    root = Node(largest_IG_feature, child_nodes)
    return root

def testWithUsedSamples(graph,data):
    num_incorrect = 0
    current_node = graph
    for ind in data.index:
        while current_node.type != "output":
            feature = current_node.feature
            next_feature = data[feature][ind]
            for j in current_node.children:
                if j.feature == next_feature:
                    current_node = j
            if len(current_node.children) == 1:
                current_node = current_node.children[0]
        label = current_node.feature
        current_node = graph
        if label != data["class"][ind]:
            num_incorrect += 1
    return num_incorrect

def drawGraph(graph, depth):
    if (len(graph.feature) != 1):
        print("Depth: "+ str(depth//2+1))
    print("feature: " + graph.feature)

    for child in graph.children:
        if len(child.children) == 1 and child.children[0].type == "output":
            print("child: "+child.feature)
            print("output: "+ child.children[0].feature)
        else:
            print("child: "+child.feature)
    print()
    for child in graph.children:
        if len(child.children) != 1 or (len(child.children) == 1 and child.children[0].type != "output"):
            drawGraph(child, (depth+1))

In [3]:

# print( testWithUsedSamples(trainGreedyDecisionTree(data), data.sample(100))/1000)
depth = 1
stopping_depth = depth*2-1
drawGraph(trainGreedyDecisionTree(data,0,stopping_depth),1)
print("----------------------------------")
depth = 2
stopping_depth = depth*2-1
drawGraph(trainGreedyDecisionTree(data,0,stopping_depth),1)
print("----------------------------------")
depth = 3
stopping_depth = depth*2-1
drawGraph(trainGreedyDecisionTree(data,0,stopping_depth),1)
print("----------------------------------")
depth = 4
stopping_depth = depth*2-1
drawGraph(trainGreedyDecisionTree(data,0,stopping_depth),1)



Depth: 1
feature: odor
child: p
output: p
child: a
output: e
child: l
output: e
child: n
output: e
child: f
output: p
child: c
output: p
child: m
output: p

----------------------------------
Depth: 1
feature: odor
child: p
output: p
child: a
output: e
child: l
output: e
child: n
child: f
output: p
child: c
output: p
child: m
output: p

feature: n
child: spore-print-color

Depth: 2
feature: spore-print-color
child: k
output: e
child: n
output: e
child: u
output: p
child: h
output: p
child: r
output: p
child: w
output: e

----------------------------------
Depth: 1
feature: odor
child: p
output: p
child: a
output: e
child: l
output: e
child: n
child: f
output: p
child: c
output: p
child: m
output: p

feature: n
child: spore-print-color

Depth: 2
feature: spore-print-color
child: k
output: e
child: n
output: e
child: u
output: p
child: h
output: p
child: r
output: p
child: w

feature: w
child: cap-color

Depth: 3
feature: cap-color
child: n
output: e
child: y
output: p
child: w
output: p

In [4]:
#Q2C
print(testWithUsedSamples(trainGreedyDecisionTree(data,0,stopping_depth), data))

0


In [5]:
#Q2D
#step 1 shuffle data
shuffled = data.sample(frac=1)
#step 2 split data
percentage_training_to_test = 0.7
training, test = np.split(shuffled, [int(percentage_training_to_test*len(data))])
#step 3 split training into training and validation
shuffled = training.sample(frac=1)
percentage_training_to_validation = 0.9
training, validation = np.split(shuffled, [int(percentage_training_to_validation*len(shuffled))])


#find best hyperparameter values
hyperparameter = {}
for j in range(1,len(training.columns)):
    stopping_depth = j

    model = trainGreedyDecisionTree(training, 0, stopping_depth*2-1)
    x = testWithUsedSamples(model,validation)
    if (x == 0 or len(validation)==0):
        hyperparameter[stopping_depth] = 0
    else:
        hyperparameter[stopping_depth] = x/(len(validation))*1.0

#choose the hyper parameter that gives the lowest validation error
best_validation = min(hyperparameter, key=hyperparameter.get)


#step 3 train model
model = trainGreedyDecisionTree(training, 0 , best_validation*2-1)
test_error = testWithUsedSamples(model,test)/(len(test))
train_error = testWithUsedSamples(model,training)/len(training)
best_error = 0 #lowest possible error
print(hyperparameter)
print("smallest validation error: "+str(hyperparameter[best_validation]))
print("training error: "+str(train_error))
print("best hyperparameter (height of tree): "+str(best_validation))
print("test error: "+str(test_error))
print("variance: "+str(test_error-train_error))
print("bias: "+str(train_error-best_error))
print("noise: "+str(best_error))
print("model error: "+str(test_error-train_error+train_error-best_error+best_error))


{1: 0.017721518987341773, 2: 0.005063291139240506, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0}
smallest validation error: 0
training error: 0.0
best hyperparameter (height of tree): 3
test error: 0.0
variance: 0.0
bias: 0.0
noise: 0
model error: 0.0
